## Estimate crop area based on crop mask (single year)

**Author**: Hannah Kerner (hkerner@umd.edu)

**Description**: This notebook performs the following steps:
1. Clips crop mask to a regional boundary (admin1 shape or user-defined bounding box)
1. Computes the confusion matrix between the labeled reference sample and the crop mask
1. Creates a random stratified sample from the crop mask for labeling in CEO
2. Calculates for the crop and noncrop area and accuracy estimates based on [Olofsson et al., 2014](https://www.sciencedirect.com/science/article/abs/pii/S0034425714000704)

To be added in the future:
- Code for thresholding the crop mask to a binary mask of 0 (noncrop) or 1 (crop)
- Code for sub-regional estimates (subsetting the reference sample according to admin2 bounds, e.g.), probably as a separate notebook

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
#import cartopy.io.shapereader as shpreader
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

## Clip crop mask to a regional boundary (admin1 shape or user-defined bounding box)

Before you begin, make sure you have taken the following steps:
1. Make sure your rasters are projected using the local UTM zone (e.g., EPSG:326XX where XX is the 2-digit UTM zone). The easiest way to reproject a raster is using gdalwarp on the command line:

`gdalwarp -t_srs <target_crs> -s_srs <source_crs> -tr 10 10 <source_filename> <dest_filename> -dstnodata 255`

2. Clip your rasters to the bounds of your region of interest using a shapefile for the region. The easiest way to do this is also using gdalwarp on the command line:

`gdalwarp -cutline <shapefile_name> -crop_to_cutline <source_filename> <dest_filename> -dstnodata -255`

If you do not have the shapefile for your ROI downloaded already, you can run the following steps to download one (note: this functionality only available for admin1 level boundaries).

If you want to use the dimensions of a bounding box instead of a shapefile, you will have the opportunity to do that later in Step 1.

In [ ]:
country_code = "CHN" # Can be found https://www.iso.org/obp/ui/#search under the Alpha-3 code column
regions_of_interest = ['Heilongjiang']

In [ ]:
# Load in shapefile from natural earth
ne_shapefile = shpreader.natural_earth(resolution='10m', category='cultural', name='admin_1_states_provinces')
ne_gdf = gpd.read_file(ne_shapefile)

if len(regions_of_interest) == 0:
    # Select entire country (all regions):
    condition = ne_gdf["adm1_code"].str.startswith(country_code)
    boundary = ne_gdf[condition].copy()
    print("Entire country found!")

else:
    # Check regions
    available_regions = ne_gdf[ne_gdf["adm1_code"].str.startswith(country_code)]["name"].tolist()
    regions_not_found = [region for region in regions_of_interest if region not in available_regions]

    if len(regions_not_found) > 0:
        condition = ne_gdf["adm1_code"].str.startswith(country_code)
        boundary = None
        print(f"WARNING: {regions_not_found} was not found. Please select regions only seen in below plot.")
    else:
        condition = ne_gdf["name"].isin(regions_of_interest)
        boundary = ne_gdf[condition].copy()
        print("All regions found!")

ne_gdf[condition].plot(
    column="name", 
    legend=True, 
    legend_kwds={'loc': 'lower right'}, 
    figsize=(10,10)
);

In [ ]:
ne_gdf.crs = 'EPSG:32xxx' # The CRS code of your map
ne_gdf.to_file('/path/to/save.shp')

## Load the crop mask

In [ ]:
with rio.open(mask_path) as src:
    if src.meta['crs'] == 'epsg:4326':
        print('''WARNING: The map CRS is EPSG:4326. This means the map unit is degrees 
              and the pixel-wise areas will not be in meters. You need to reproject the map
              to the projection defined for the map's primary UTM zone (e.g., EPSG:32652).''')
    if src.meta['dtype'] != 'uint8':
        print('''WARNING: The map data type is %s but should be uint8. Make sure the map has
              been thresholded to convert to a binary mask of 0 (noncrop) or 1 (crop).''')
    else:
        print('Map CRS is %s. Loading map into memory.' % src.crs)
        crop_map = src.read(1).astype(np.uint8)

In [ ]:
# Loads the raster from the .tif file as a numpy array
# Returns a masked array where the nodata values are masked
def load_raster(path, target_crs='epsg:4326', clip_coords=None):
    with rio.open(path) as src:
        if src.meta['crs'] == 'epsg:4326':
            print('''WARNING: The map CRS is EPSG:4326. This means the map unit is degrees 
                  and the pixel-wise areas will not be in meters. You need to reproject the map
                  to the projection defined for the map's primary UTM zone (e.g., EPSG:32652).''')
        else:
            print('Map CRS is %s. Loading map into memory.' % src.crs)
            print(src.meta)
            if clip_coords:
                raster, out_transform = mask(src, shapes=clip_coords, crop=True)
                raster = raster[0]
                # Update the metadata
                out_meta = src.meta.copy()
                out_meta.update({"driver": "GTiff",
                                 "height": raster.shape[0],
                                 "width": raster.shape[1],
                                 "transform": out_transform,
                                 "crs": src.meta['crs']}
                               )
                return np.ma.masked_equal(raster, src.meta['nodata']), out_meta
            else:
                raster = src.read(1).astype(np.uint8)
                return np.ma.masked_equal(raster, src.meta['nodata']), src.meta

In [ ]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

# Optionally specify bounding box boundaries to clip to
# Note that these boundaries must be in the same CRS as the raster
# You can get this from bboxfinder, e.g.: http://bboxfinder.com/#10.277000,36.864900,10.835100,37.191000
minx, miny, maxx, maxy = # your optional bbox bounds, e.g.
                         # 249141.6217,840652.3433,272783.1953,855138.2342
bbox = box(minx, miny, maxx, maxy)
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=target_crs)
coords = getFeatures(geo)

In [ ]:
mask_path = # path to your binary map file, 
            # e.g. '/gpfs/data1/cmongp1/barker/binary_masks/reprojected/epsg32652_HLJ_2019.tif'
    
target_crs = 'epsg:xxxxx' # the CRS you want to use (see discussion about CRS in step 1 below)

In [ ]:
crop_map, crop_map_meta = load_raster(mask_path, target_crs)

In [ ]:
# Plot the map to make sure it looks as expected
# This may take a while depending on the size of the map,
# so you may choose not to run this every time.
# plt.imshow(crop_map, cmap='YlGn');
# plt.axis('off');

## Calculate the mapped area for each class

In [ ]:
pixel_size = src.transform[0]
print('The pixel size is %f meters.' % pixel_size)

In [ ]:
# Function to calculate mapped area in pixels or ha
def mapped_area(pred_map, unit='pixels', px_size=10):
    crop_px = np.where(pred_map.flatten() == 1)
    noncrop_px = np.where(pred_map.flatten() == 0)
    total = crop_px[0].shape[0] + noncrop_px[0].shape[0]
    if unit == 'ha':
        # Multiply pixels by area per pixel and convert m to hectares
        crop_area = crop_px[0].shape[0] * (px_size*px_size) / 100000
        noncrop_area = noncrop_px[0].shape[0] * (px_size*px_size) / 100000
    elif unit == 'pixels':
        crop_area = int(crop_px[0].shape[0])
        noncrop_area = int(noncrop_px[0].shape[0])
    elif unit == 'fraction':
        crop_area = crop_px[0].shape[0] / total
        noncrop_area = noncrop_px[0].shape[0] / total
        assert (crop_area + noncrop_area) == 1
    return crop_area, noncrop_area

In [ ]:
crop_area_frac, noncrop_area_frac = mapped_area(crop_map, unit='fraction')

print('Crop area [fraction] = %f' % crop_area_frac)
print('Non-crop area [fraction] = %f' % noncrop_area_frac)

In [ ]:
crop_area_px, noncrop_area_px = mapped_area(crop_map)

print('Crop area [pixels] = %d' % crop_area_px)
print('Non-crop area [pixels] = %d' % noncrop_area_px)

In [ ]:
tot_area_px = crop_area_px + noncrop_area_px
print('Total area [pixels] = %d' % tot_area_px)

In [ ]:
crop_area_ha, noncrop_area_ha = mapped_area(crop_map, unit='ha', px_size=pixel_size)

print('Crop area [ha] = %d' % crop_area_ha)
print('Non-crop area [ha] = %d' % noncrop_area_ha)

## Create random stratified reference sample from change map strata following best practices

First we need to determine the number of total samples we want to label for our reference dataset.

We use the method identified by Olofsson et al. in [*Good practices for estimating area and assessing accuracy of land change*](https://www.sciencedirect.com/science/article/pii/S0034425714000704) (eq 13) to determine sample size:

n ≈ ( Σ(W<sub>i</sub>S<sub>i</sub>) / S(Ô) )<sup>2</sup>

| Where         |                                                      |
|---------------|------------------------------------------------------|
| W<sub>i</sub> | Mapped proportion of class i                         |
| S<sub>i</sub> | Standard deviation √(U<sub>i</sub>(1-U<sub>i</sub>)) |
| U<sub>i</sub> | Expected user's accuracy for class i                 |
| S(Ô)          | Standard error                                       |
| n             | Sample size                                          |


If you have already used an independent validation or test set to estimate the user's accuracy for each class, you can plug those values into this equation. If you have not already calculated it, you will need to make a guess (it is better to make a conservative guess since an overestimation may lead to fewer points than are actually needed to achieve low standard errors). See the example calculation below for user's accuracy of both classes of 0.63 and a standard error of 0.02.

In [ ]:
u_crop = 0.7
u_noncrop = 0.7
stderr = 0.02

s_crop = np.sqrt(u_crop * (1-u_crop))
s_noncrop = np.sqrt(u_noncrop * (1-u_noncrop))

n = np.round(((crop_area_frac*s_crop + noncrop_area_frac*s_noncrop) / stderr)**2)

print('Num samples: {}'.format(int(n)))

We will use a stratified random sample, meaning we will sample some number of points randomly within each of our map strata (the crop and non-crop classes). Refer to Section 5.1.2 in Olofsson et al. for in depth discussion on different sample allocation methods and choose the one that best fits your use case. Below, we use the equal allocation strategy to allocate an approximately equal number of samples to the crop and non-crop map strata.

In [ ]:
n_crop = int(n / 2)
n_noncrop = int(n - n_crop)

In [ ]:
print('Sample allocation:')
print('Crop: {}'.format(n_crop))
print('Non-crop: {}'.format(n_noncrop))

Now we can randomly draw sample locations using this allocation from each of the map strata.

In [ ]:
def random_inds(raster, strata, n):
    inds = np.where(raster == strata)
    rand_inds = np.random.choice(np.arange(inds[0].shape[0]), size=n, replace=False)
    rand_px = inds[0][rand_inds]
    rand_py = inds[1][rand_inds]
    return rand_px, rand_py

In [ ]:
df_noncrop = pd.DataFrame([], columns=['px', 'py', 'pred_class'])
df_noncrop['px'], df_noncrop['py'] = random_inds(crop_map, 0, n_noncrop)
df_noncrop['pred_class'] = 0

In [ ]:
df_crop = pd.DataFrame([], columns=['px', 'py', 'pred_class'])
df_crop['px'], df_crop['py'] = random_inds(crop_map, 1, n_crop)
df_crop['pred_class'] = 1

In [ ]:
df_combined = pd.concat([df_noncrop, df_crop]).reset_index(drop=True)

In [ ]:
# Plot the number of samples in each strata to confirm
df_combined.groupby('pred_class').count()['px'].plot(kind='bar')

In [ ]:
# Convert the pixel locations to geographic locations
with rio.open(mask_path) as src:
    for r, row in df_combined.iterrows():
        # translate to geographic coordinates
        lx, ly = src.xy(row['px'], row['py'])
        df_combined.loc[r, 'lx'] = lx
        df_combined.loc[r, 'ly'] = ly

In [ ]:
# Shuffle the dataframe so the classes are randomized in CEO
df_combined = df_combined.sample(frac=1).reset_index(drop=True)

In [ ]:
df_combined.head()

In [ ]:
# Convert the pandas dataframe to a geopandas dataframe
gdf = gpd.GeoDataFrame(df_combined, geometry=gpd.points_from_xy(df_combined.lx, df_combined.ly))

In [ ]:
gdf.head()

In [ ]:
gdf.crs = target_crs

In [ ]:
# Save the labels in the local CRS first
ref_sample_path_local = '/your/file/path.shp'

gdf.to_file(ref_sample_path_local)

In [ ]:
# Convert to 4326 for CEO
gdf_4326 = gdf.to_crs('EPSG:4326')

In [ ]:
gdf_4326['PLOTID'] = gdf_4326.index

In [ ]:
gdf_4326['SAMPLEID'] = gdf_4326.index

In [ ]:
gdf_4326.head()

In [ ]:
# Save the labels in the CEO format
ref_sample_path_ceo = '/your/file/path.shp'
gdf_4326[['geometry', 'PLOTID', 'SAMPLEID']].to_file(ref_sample_path_ceo, index=False)

### Label the reference samples in CEO

This step is done in Collect Earth Online. First you need to create a labeling project with the shapefile we just created (two copies for consensus). Once all of the points in both sets have been labeled, come back to the next step.

See the instructions for labeling planted area points [here](https://docs.google.com/presentation/d/18bJHMX5M1jIR9NBWIdYeJyo3tG4CL3dNO5vvxOpz5-4/edit#slide=id.p).

## 4. Load the labeled reference samples

There should be two sets of labels for the reference sample. We compare the labels from each set to filter out labels for which the labelers did not agree and thus we can't be confident about the true label.

In [ ]:
ceo_set1_path = # path to your CEO Set 1 CSV file, 
                # e.g. '/gpfs/data1/cmongp1/barker/labeled_ceo/ceo-HLJ-2019-(Set-1)---v3-sample-data-2022-01-21.csv'

ceo_set2_path = # path to your CEO Set 2 CSV file, 
                # e.g. '/gpfs/data1/cmongp1/barker/labeled_ceo/ceo-HLJ-2019-(Set-2)---v3-sample-data-2022-01-21.csv'

In [ ]:
ceo_set1 = pd.read_csv(ceo_set1_path)
ceo_set1.head()

In [ ]:
ceo_set2 = pd.read_csv(ceo_set2_path)
ceo_set2.head()

In [ ]:
# Sometimes there are slight variations in the labeling question used. We store it here to avoid manual updates.
label_question = ceo_set1.columns[-1]

In [ ]:
ceo_set1.shape[0]

In [ ]:
ceo_set1[ceo_set1[label_question] == 'Crop']

In [ ]:
# Check for any NaNs / missing answers
ceo_set1[ceo_set1[label_question].isna()]

In [ ]:
# Check for any NaNs / missing answers
ceo_set2[ceo_set2[label_question].isna()]

In [ ]:
if ceo_set1.shape != ceo_set2.shape:
    print('ERROR: The size of the two dataframes does not match. Most likely, there is a duplicate in the plotid column resulting from an error in CEO. You need to delete the duplicate manually before continuing.')
    print(ceo_set1[ceo_set1.duplicated(subset=['plotid'])])
    print(ceo_set2[ceo_set2.duplicated(subset=['plotid'])])

In [ ]:
# Make sure the question and thus column name is correct for the project you are working on
ceo_agree = ceo_set1[ceo_set1[label_question] == 
                         ceo_set2[label_question]]

print('Number of samples that are in agreement: %d out of %d (%.2f%%)' % 
          (ceo_agree.shape[0], ceo_set1.shape[0], ceo_agree.shape[0]/ceo_set1.shape[0]*100))

In [ ]:
# Convert the pandas dataframe to a geodataframe
ceo_agree_geom = gpd.GeoDataFrame(ceo_agree, geometry=gpd.points_from_xy(ceo_agree.lon, ceo_agree.lat), crs='EPSG:4326')

In [ ]:
# The labeling platform CEO requires points to be in EPSG:4326. 
# Reproject to the same crs as the map.
ceo_agree_geom = ceo_agree_geom.to_crs(src.crs)

In [ ]:
# Plot them to make sure they look as expected
ceo_agree_geom.plot();

## 5. Get the mapped class for each of the reference samples

In [ ]:
for r, row in ceo_agree_geom.iterrows():
    # transform lon, lat to pixel coordinates
    lon, lat = row['geometry'].y, row['geometry'].x
    px, py = src.index(lat, lon)
    ceo_agree_geom.loc[r,'Mapped class'] = crop_map[px, py]

In [ ]:
ceo_agree_geom.head()

In [ ]:
# Make sure none of them are nodata
ceo_agree[ceo_agree_geom['Mapped class'] == 3]

## 6. Compute the confusion matrix between the mapped classes and reference labels

In [ ]:
# Convert the CEO string label to an integer label
ceo_agree_geom.loc[ceo_agree_geom[label_question] == 'Crop', 'Reference label'] = 1
ceo_agree_geom.loc[ceo_agree_geom[label_question] == 'Non-crop', 'Reference label'] = 0
# Account for alternate spellings that may be pressent
ceo_agree_geom.loc[ceo_agree_geom[label_question] == 'Noncrop', 'Reference label'] = 0
ceo_agree_geom.loc[ceo_agree_geom[label_question] == 'NonCrop', 'Reference label'] = 0

ceo_agree_geom['Reference label'] = ceo_agree_geom['Reference label'].astype(np.uint8)
ceo_agree_geom.head()

In [ ]:
# Compute confusion matrix
y_true = np.array(ceo_agree_geom['Reference label']).astype(np.uint8)
y_pred = np.array(ceo_agree_geom['Mapped class']).astype(np.uint8)
confusion_matrix(y_true, y_pred)

In [ ]:
# Extract and print confusion matrix values with element descriptions
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print('True negatives: %d' % tn)
print('False positives: %d' % fp)
print('False negatives: %d' % fn)
print('True positives: %d' % tp)

## 7. Adjust mapped area using confusion matrix to compute area estimates

$W_h$ is the proportion of mapped area for each class 

In [ ]:
wh_crop = crop_area_px / tot_area_px
print('Wh_crop = %f' % wh_crop)

wh_noncrop = noncrop_area_px / tot_area_px
print('Wh_noncrop = %f' % wh_noncrop)

Compute the fraction of the proportional area of each class that was mapped as each category in the confusion matrix

In [ ]:
tp_area = tp / (tp + fp) * wh_crop
fp_area = fp / (tp + fp) * wh_crop
fn_area = fn / (fn + tn) * wh_noncrop
tn_area = tn / (fn + tn) * wh_noncrop

print('%f \t %f \n %f \t %f' % (tp_area, fp_area, fn_area, tn_area))

$U_i$ is the user's accuracy (i.e., precision) for each mapped class. We calculate it here in terms of proportion of area computed in the last cell.

In [ ]:
u_crop = tp_area / (tp_area + fp_area)
print('U_crop = %f' % u_crop)

u_noncrop = tn_area / (tn_area + fn_area)
print('U_noncrop = %f' % u_noncrop)

$V(U_i)$ is the estimated variance of user accuracy for each mapped class.

In [ ]:
v_u_crop = u_crop * (1-u_crop) / (tp + fp)
print('V(U)_crop = %f' % v_u_crop)

v_u_noncrop = u_noncrop * (1-u_noncrop) / (fn + tn)
print('V(U)_noncrop = %f' % v_u_noncrop)

$S(U_i)$ is the estimated standard error of user accuracy for each mapped class.

In [ ]:
s_u_crop = np.sqrt(v_u_crop)
print('S(U)_crop = %f' % s_u_crop)

s_u_noncrop = np.sqrt(v_u_noncrop)
print('S(U)_noncrop = %f' % s_u_noncrop)

Get the 95% confidence interval for User's accuracy

In [ ]:
u_crop_err = s_u_crop * 1.96
print('95%% CI of User accuracy for crop = %f' % u_crop_err)

u_noncrop_err = s_u_noncrop * 1.96
print('95%% CI of User accuracy for noncrop = %f' % u_noncrop_err)

$P$ is the producer's accuracy (i.e., recall). We calculate it here in terms of proportion of area.

In [ ]:
p_crop = tp_area / (tp_area + fn_area)
print('P_crop = %f' % p_crop)

p_noncrop = tn_area / (tn_area + fp_area)
print('P_noncrop = %f' % p_noncrop)

$N_j$ is the estimated marginal total number of pixels of each reference class $j$

In [ ]:
n_j_crop = (crop_area_px * tp) / (tp + fp) + (noncrop_area_px * fn) / (fn + tn)
print('N_j_crop = %f' % n_j_crop)

n_j_noncrop = (crop_area_px * fp) / (tp + fp) + (noncrop_area_px * tn) / (fn + tn)
print('N_j_crop = %f' % n_j_noncrop)

In [ ]:
expr1_crop = crop_area_px**2 * (1-p_crop)**2 * u_crop * (1-u_crop) / (tp + fp - 1)
print('expr1 crop = %f' % expr1_crop)

expr1_noncrop = noncrop_area_px**2 * (1-p_noncrop)**2 * u_noncrop * (1-u_noncrop) / (fp + tn - 1)
print('expr1 noncrop = %f' % expr1_noncrop)

In [ ]:
# Warning: depending on the size of your map, you may get an overflow warning here, e.g.
# RuntimeWarning: overflow encountered in long_scalars

expr2_crop = p_crop**2 * (noncrop_area_px**2 * fn / (fn + tn) * (1 - fn / (fn + tn)) / (fn + tn - 1))
print('expr2 crop = %f' % expr2_crop)

expr2_noncrop = p_crop**2 * (crop_area_px**2 * fp / (fp + tp) * (1 - fp / (fp + tp)) / (fp + tp - 1))
print('expr2 noncrop = %f' % expr2_noncrop)

$V(P_i)$ is the estimated variance of producer's accuracy for each mapped class.

In [ ]:
v_p_crop = (1 / n_j_crop**2) * (expr1_crop + expr2_crop)
print('V(P) crop = %f' % v_p_crop)

v_p_noncrop = (1 / n_j_noncrop**2) * (expr1_noncrop + expr2_noncrop)
print('V(P) noncrop = %f' % v_p_noncrop)

$S(P_i)$ is the estimated standard error of producer accuracy for each mapped class.

In [ ]:
# Warning: depending on the size of your map, you may get an overflow warning here, e.g.
# RuntimeWarning: overflow encountered in long_scalars

s_p_crop = np.sqrt(v_p_crop)
print('S(P) crop = %f' % s_p_crop)

s_p_noncrop = np.sqrt(v_p_noncrop)
print('S(P) noncrop = %f' % s_p_noncrop)

Get the 95% confidence interval for Producer's accuracy

In [ ]:
p_crop_err = s_p_crop * 1.96
print('95%% CI of Producer accuracy for crop = %f' % p_crop_err)

p_noncrop_err = s_p_noncrop * 1.96
print('95%% CI of Producer accuracy for noncrop = %f' % p_noncrop_err)

$O$ is the overall accuracy. We calculate it here in terms of proportion of area.

In [ ]:
acc = tp_area + tn_area
print('Overall accuracy = %f' % acc)

$V(O)$ is the estimated variance of the overall accuracy

In [ ]:
v_acc = wh_crop**2 * u_crop * (1-u_crop) / (tp + fp - 1) + \
        wh_noncrop**2 * u_noncrop * (1-u_noncrop) / (fn + tn - 1)
print('V(O) = %f' % v_acc)

$S(O)$ is the estimated standard error of the overall accuracy

In [ ]:
s_acc = np.sqrt(v_acc)
print('S(O) = %f' % s_acc)

Get the 95% confidence interval for overall accuracy

In [ ]:
acc_err = s_acc * 1.96
print('95%% CI of overall accuracy = %f' % acc_err)

$A_{pixels}$ is the adjusted map area in units of pixels

In [ ]:
a_pixels_crop = tot_area_px * (tp_area + fn_area)
print('A^[pixels] crop = %f' % a_pixels_crop)

a_pixels_noncrop = tot_area_px * (tn_area + fp_area)
print('A^[pixels] noncrop = %f' % a_pixels_noncrop)

$A_{ha}$ is the adjusted map area in units of hectares

In [ ]:
a_ha_crop = a_pixels_crop * (pixel_size*pixel_size) / (100*100)
print('A^[ha] crop = %f' % a_ha_crop)

a_ha_noncrop = a_pixels_noncrop * (pixel_size*pixel_size) / (100*100)
print('A^[ha] noncrop = %f' % a_ha_noncrop)

The following equations are used to estimate the standard error for the area. They are based on the calculations in Olofsson et al., 2014.

In [ ]:
S_pk_crop = np.sqrt((wh_crop * tp_area - tp_area**2) / (tp + fp - 1) + \
                     (wh_noncrop * fn_area - fn_area**2) / (fn + tn - 1)) * tot_area_px
print('S_pk_crop = %f' % S_pk_crop)

S_pk_noncrop = np.sqrt((wh_crop * fp_area - fp_area**2) / (tp + fp - 1) + \
                        (wh_noncrop * tn_area - tn_area**2) / (fn + tn - 1)) * tot_area_px
print('S_pk_noncrop = %f' % S_pk_noncrop)

Multiply $S(p_k)$ by 1.96 to get the margin of error for the 95% confidence interval

In [ ]:
a_pixels_crop_err = S_pk_crop * 1.96
print('Crop area standard error 95%% confidence interval [pixels] = %f' % a_pixels_crop_err)

a_pixels_noncrop_err = S_pk_noncrop * 1.96
print('Non-crop area standard error 95%% confidence interval [pixels] = %f' % a_pixels_noncrop_err)

In [ ]:
a_ha_crop_err = a_pixels_crop_err * (pixel_size**2) / (100**2)
print('Crop area standard error 95%% confidence interval [ha] = %f' % a_ha_crop_err)

a_ha_noncrop_err = a_pixels_noncrop_err * (pixel_size**2) / (100**2)
print('Non-crop area standard error 95%% confidence interval [ha] = %f' % a_ha_noncrop_err)

Summary of the final estimates of accuracy and area with standard error at 95% confidence intervals:

In [ ]:
summary = pd.DataFrame([[a_ha_crop, a_ha_noncrop],
                        [a_ha_crop_err, a_ha_noncrop_err],
                        [u_crop, u_noncrop],
                        [u_crop_err, u_noncrop_err],
                        [p_crop, p_noncrop],
                        [p_crop_err, p_noncrop_err],
                        [acc, acc],
                        [acc_err, acc_err]
                       ],
                       index=pd.Index(['Estimated area [ha]', '95% CI of area [ha]', 'User accuracy',
                                       '95% CI of user acc', 'Producer accuracy', '95% CI of prod acc',
                                       'Overall accuracy', '95% CI of overall acc']),
                       columns=['Crop', 'Non-crop'])

summary.round(2)